In [35]:
import cv2
import numpy as np
import csv
import os

# Load calibration parameters
camera_matrix = np.array([[547.78085475,   0.        , 358.73466416],
                          [  0.        , 548.79981379, 285.09965897],
                          [  0.        ,   0.        ,   1.        ]], dtype=float)
dist_coeffs = np.array([[-0.0336199 ,  1.2924381 ,  0.01657285,  0.0244749 , -3.99229428]], dtype=float)

# Define the output CSV file path
csv_file_path = r'C:\Users\shann\OneDrive\Documents\Lake restoration\track_boat.csv'

# Function to write the header to the CSV file
def write_header(file_path):
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['x', 'y', 'z'])  # Write header

# Function to write the coordinates to the CSV file
def write_coordinates(file_path, x, y, z):
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([x, y, z])

def track(matrix_coefficients, distortion_coefficients):
    cap = cv2.VideoCapture(0)  # Initialize the webcam
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Define the 3D points of the ARUCO marker's corners in its coordinate system
    marker_length = 0.22  # Marker length in meters
    marker_3D_points = np.array([
        [-marker_length / 2, marker_length / 2, 0],
        [marker_length / 2, marker_length / 2, 0],
        [marker_length / 2, -marker_length / 2, 0],
        [-marker_length / 2, -marker_length / 2, 0]
    ], dtype=np.float32)

    # Always write a new header to the CSV file, effectively rewriting the file each time
    write_header(csv_file_path)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

        aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_1000)  # Use 5x5 dictionary to find markers
        parameters = cv2.aruco.DetectorParameters()  # Marker detection parameters

        # Detect markers
        corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
        if np.all(ids is not None):  # If there are markers found by detector
            for i in range(len(ids)):  # Iterate over markers
                # Get the 2D corners of the detected marker
                marker_2D_corners = corners[i][0]

                # Estimate pose of each marker using solvePnP
                success, rvec, tvec = cv2.solvePnP(marker_3D_points, marker_2D_corners, matrix_coefficients, distortion_coefficients)
                if success:
                    cv2.aruco.drawDetectedMarkers(frame, corners)  # Draw a square around the markers
                    cv2.drawFrameAxes(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.11)  # Draw Axis

                    # Marker position in camera coordinates
                    marker_position_camera = tvec.T[0]

                    # Write coordinates to CSV
                    write_coordinates(csv_file_path, marker_position_camera[0], marker_position_camera[1], marker_position_camera[2])

                    # Print the Cartesian coordinates of the marker
                    print(f"Marker Cartesian Coordinates: x={marker_position_camera[0]}, y={marker_position_camera[1]}, z={marker_position_camera[2]}")

        # Display the resulting frame
        cv2.imshow('frame', frame)
        # Wait 3 milliseconds for an interaction. Check the key and do the corresponding job.
        key = cv2.waitKey(3) & 0xFF
        if key == ord('q'):  # Quit
            break

    # When everything is done, release the capture
    cap.release()
    cv2.destroyAllWindows()

# Call the function with calibration parameters
track(camera_matrix, dist_coeffs)


Marker Cartesian Coordinates: x=0.8897469016301934, y=0.3833627044416979, z=1.9335085911577603
Marker Cartesian Coordinates: x=0.8599945418059891, y=0.36633185802479845, z=1.8857842895836268
Marker Cartesian Coordinates: x=0.8557392333900473, y=0.3657144649630556, z=1.8782143259250619
Marker Cartesian Coordinates: x=0.8815804369870635, y=0.36632462299310914, z=1.9603991630947013
Marker Cartesian Coordinates: x=0.8700809893326561, y=0.36300219011038476, z=1.9424975163552032
Marker Cartesian Coordinates: x=1.013566729571411, y=0.41062976064439405, z=2.2759036326902686
Marker Cartesian Coordinates: x=1.0273855356004293, y=0.41231450276252823, z=2.329774467074495
Marker Cartesian Coordinates: x=0.9976349198095067, y=0.4041354710471421, z=2.2669082349995877
Marker Cartesian Coordinates: x=1.027487360642304, y=0.41383086151582854, z=2.3439483063500597
Marker Cartesian Coordinates: x=0.9571670523028093, y=0.384835745644445, z=2.3006714119578713
Marker Cartesian Coordinates: x=0.95237243312976